# Описание проекта

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. 
Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».
Компания провела опрос и попросила вас подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite. 
Чтобы оценить результаты опроса, оценки обычно делят на три группы:
9-10 баллов — «cторонники» (англ. promoters);
7-8 баллов — «нейтралы» (англ. passives);
0-6 баллов — «критики» (англ. detractors).
Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».
Таким образом, значение этого показателя варьируется от -100% (когда все клиенты «критики») до 100% (когда все клиенты лояльны к сервису). Но это крайние случаи, которые редко встретишь на практике. 
Интерпретируя результаты NPS-опросов, следует также помнить, что само значение мало о чём говорит. Однако исследования показывают, что клиенты-сторонники полезны любому бизнесу. Они чаще других повторно совершают покупки, активнее тестируют обновления и приводят в сервис своих друзей и знакомых. Поэтому NPS остаётся одной из важнейших метрик бизнеса.

# Описание данных 


- user_id - Идентификатор клиента
- lt_day - Количество дней «жизни» клиента
- is_new - Поле хранит информацию о том, является ли клиент новым
- age - Возраст
- gender_segment - Пол 
- os_name - Тип операционной системы
- cpe_type_name - Тип устройства
- country - Страна проживания
- city - Город проживания
- age_segment - Возрастной сегмент
- traffic_segment - Сегмент по объёму потребляемого трафика
- lifetime_segment - Сегмент по количеству дней «жизни»
- nps_score - Оценка клиента в NPS-опросе
- nps_group - Поле хранит информацию о том, к какой группе относится оценка клиента в опросе

# Вопросы исследования

- Как распределены участники опроса по возрасту и полу? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?
- Какие группы пользователей наиболее лояльны к сервису? Какие менее?
- Какой общий NPS среди всех опрошенных?
- Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?

# Структура исследования

- Загрузка баз данных
- Предпроспорт имеющихся баз данных
- Предобработка данных и объединение
- Презентация Tableau

## Загрузка баз данных

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

## Предпросмотр баз данных

In [3]:
query = """
SELECT * 
FROM user;
"""
view = pd.read_sql(query, engine)
view

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,nps_score
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,55,5,5,8,10
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,21,5,5,8,10
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,28,6,9,6,10
3,A004TB,4190,44.0,1.0,IOS,SMARTPHONE,38,4,4,8,10
4,A004XT,1163,24.0,0.0,ANDROID,SMARTPHONE,39,2,6,8,10
...,...,...,...,...,...,...,...,...,...,...,...
502488,ZZZKLD,1249,54.0,1.0,ANDROID,SMARTPHONE,28,5,5,8,5
502489,ZZZLWY,129,31.0,0.0,ANDROID,SMARTPHONE,28,3,5,4,8
502490,ZZZQ5F,522,36.0,0.0,ANDROID,SMARTPHONE,47,4,10,6,10
502491,ZZZQ8E,2936,37.0,1.0,ANDROID,SMARTPHONE,53,4,18,8,9


In [4]:
query = """
        SELECT * 
        FROM location;
        """
view = pd.read_sql(query, engine)
view

,location_id,city,country
0,1,Архангельск,Россия
1,2,Астрахань,Россия
2,3,Балашиха,Россия
3,4,Барнаул,Россия
4,5,Белгород,Россия
...,...,...,...
57,58,Челябинск,Россия
58,59,Череповец,Россия
59,60,Чита,Россия
60,61,Якутск,Россия


In [5]:
for i in range(len(view)):
    print(view.iloc[i,1])

Архангельск
Астрахань
Балашиха
Барнаул
Белгород
Брянск
Владивосток
Владимир
Волгоград
Волжский
Воронеж
Грозный
Екатеринбург
Иваново
Ижевск
Иркутск
Казань
Калининград
Калуга
Кемерово
Киров
Краснодар
Красноярск
Курск
Липецк
Магнитогорск
Махачкала
Москва
НабережныеЧелны
НижнийНовгород
НижнийТагил
Новокузнецк
Новосибирск
Омск
Оренбург
Пенза
Пермь
РостовнаДону
Рязань
Самара
СанктПетербург
Саранск
Саратов
Смоленск
Сочи
Ставрополь
Сургут
Тверь
Тольятти
Томск
Тула
Тюмень
УланУдэ
Ульяновск
Уфа
Хабаровск
Чебоксары
Челябинск
Череповец
Чита
Якутск
Ярославль


**Промежуточный вывод:** В названиях некоторых городов видны ошибки/опечатки при написании. 

In [6]:
query = """
        SELECT SUBSTR(title, 3) AS age_segment
        FROM age_segment;
        """
view = pd.read_sql(query, engine)
view

,age_segment
0,до 16
1,16-24
2,25-34
3,35-44
4,45-54
5,55-64
6,66 +
7,n/a


In [7]:
query = """
        SELECT * 
        FROM lifetime_segment;
        """
view = pd.read_sql(query, engine)
view

,lt_gr_id,bucket_min,bucket_max,title
0,1,1.0,1.0,01 1
1,2,2.0,2.0,02 2
2,3,3.0,3.0,03 3
3,4,4.0,6.0,04 4-6
4,5,7.0,12.0,05 7-12
5,6,13.0,24.0,06 13-24
6,7,25.0,36.0,07 25-36
7,8,36.0,NaN,08 36+


**Вывод:** в представленных базах данных есть ошибки в написании некоторых параметров. Они будут исправлены в ходе дальнейшей работы. 

## Объединение баз данных и предобработка данных

Составим запрос, в котором исправим замеченые на предпросмотре ошибки. Также будут сформированы категориальные значения для более удобной работы в дальнейшем. 

In [8]:
query = """
SELECT user_id,
       lt_day,
       CASE
           WHEN lt_day <= 365 THEN 'Новый'
           WHEN lt_day >= 365 THEN 'Старый'
       END AS is_new, 
       age,
       CASE
           WHEN gender_segment = 0 THEN 'Мужчина'
           WHEN gender_segment = 1 THEN 'Женщина'
           ELSE 'неопределён'
       END AS gender_segment,
       os_name,
       cpe_type_name,
       loc.country,
       CASE
               WHEN loc.city = 'НабережныеЧелны' THEN 'Набережные Челны'
               WHEN loc.city = 'НижнийНовгород' THEN 'Нижний Новгород'
               WHEN loc.city = 'НижнийТагил' THEN 'Нижний Тагил'
               WHEN loc.city = 'РостовнаДону' THEN 'Ростов-на-Дону'
               WHEN loc.city = 'СанктПетербург' THEN 'Санкт-Петербург'
               WHEN loc.city = 'УланУдэ' THEN 'Улан-Удэ'
               ELSE loc.city 
       END AS city,
       SUBSTR(age_segment.title, 4) AS age_segment,
       SUBSTR(traffic_segment.title, 4) AS traffic_segment,
       SUBSTR(lifetime_segment.title, 4) AS lifetime_segment,
       nps_score,
       CASE
           WHEN nps_score >= 9 THEN 'сторонники'
           WHEN nps_score > 6 AND nps_score < 9 THEN 'нейтралы'
           WHEN nps_score <= 6 THEN 'критики'
       END AS nps_group
       
FROM user

INNER JOIN location AS loc ON user.location_id=loc.location_id
INNER JOIN age_segment ON user.age_gr_id = age_segment.age_gr_id
INNER JOIN lifetime_segment ON user.lt_gr_id = lifetime_segment.lt_gr_id
INNER JOIN traffic_segment ON user.tr_gr_id = traffic_segment.tr_gr_id
"""

In [9]:
df = pd.read_sql(query, engine)
df.head(20)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,Старый,45.0,Женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,сторонники
1,A001WF,2344,Старый,53.0,Мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,сторонники
2,A003Q7,467,Старый,57.0,Мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,сторонники
3,A004TB,4190,Старый,44.0,Женщина,IOS,SMARTPHONE,Россия,Ростов-на-Дону,35-44,0.1-1,36+,10,сторонники
4,A004XT,1163,Старый,24.0,Мужчина,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,сторонники
5,A005O0,5501,Старый,42.0,Женщина,ANDROID,SMARTPHONE,Россия,Омск,35-44,5-10,36+,6,критики
6,A0061R,1236,Старый,45.0,Мужчина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,10-15,36+,10,сторонники
7,A009KS,313,Новый,35.0,Мужчина,ANDROID,SMARTPHONE,Россия,Москва,35-44,45-50,7-12,10,сторонники
8,A00AES,3238,Старый,36.0,Женщина,ANDROID,SMARTPHONE,Россия,Санкт-Петербург,35-44,1-5,36+,10,сторонники
9,A00F70,4479,Старый,54.0,Женщина,ANDROID,SMARTPHONE,Россия,Волгоград,45-54,15-20,36+,9,сторонники


In [10]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

**Вывод:** базы данных объеденены, исправлены значения, добавлены категориальные параметры. 

## Презентация Tableau

Ссылка на презентацию: https://public.tableau.com/views/NPSTelecom_17283040037230/TELECOMNPSPRESENTATION?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link